<a href="https://colab.research.google.com/github/komaltp/project/blob/main/English_Grammar_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm
import csv

In [ ]:
df= pd.read_csv("/content/Sentence pairs in English-English - 2022-08-17 (1).csv")
df.columns = ["enc_input","dec_input","y"] 
df["dec_output"] = df.dec_input
df

,enc_input,dec_input,y,dec_output
0,I we be back soon.,I will be back soon.,1.0,I will be back soon.
1,I will e back soon.,I will be back soon.,1.0,I will be back soon.
2,u are in my way.,You are in my way.,1.0,You are in my way.
3,I might give up soon and just nap instead.,I may give up soon and just nap instead.,1.0,I may give up soon and just nap instead.
4,Are u sure?,Are you sure?,1.0,Are you sure?
...,...,...,...,...
5136,"If the plug becomes dirty, wipe it off with a ...","If the plug becomes dirty, wipe it off with a ...",NaN,"If the plug becomes dirty, wipe it off with a ..."
5137,She slammed the door shut.,She slammed the door shut.,NaN,She slammed the door shut.
5138,My father's going to kill me.,My father's going to kill me.,NaN,My father's going to kill me.
5139,It's popular among senior citizens.,It's popular among senior citizens.,NaN,It's popular among senior citizens.


In [ ]:
df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>"

In [ ]:
df_sampled = pd.concat((df[df.y==1].sample(frac= 0.2,random_state=1),df[df.y==1]))
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST

df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3, stratify = df_sampled.y )
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER

df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

In [ ]:
df_sampled

,enc_input,dec_input,y,dec_output
457,I don't got it.,<start> I don't get it.,1.0,I don't get it. <end>
148,Communism was the system practiced in the Sovi...,<start> Communism is the system practiced in t...,1.0,Communism is the system practiced in the Sovie...
741,Thiss door won't open.,<start> This door won't open.,1.0,This door won't open. <end>
526,It is a matter of lifee or death.,<start> It is a matter of life or death.,1.0,It is a matter of life or death. <end>
400,Mr Brown alwayed carries a book with him.,<start> Mr Brown always carries a book with him.,1.0,Mr Brown always carries a book with him. <end>
...,...,...,...,...
759,Ken couldn't recall has name.,<start> Ken couldn't recall his name.,1.0,Ken couldn't recall his name. <end>
760,Ken appears an friendly person.,<start> Ken appears a friendly person.,1.0,Ken appears a friendly person. <end>
761,Shall I ccall Ken back?,<start> Shall I call Ken back?,1.0,Shall I call Ken back? <end>
762,kate iis no less charming than her sister is.,<start> Kate is no less charming than her sist...,1.0,Kate is no less charming than her sister is. <...


In [ ]:
df_val

,enc_input,dec_input,y,dec_output
719,This worked doesn't pay.,<start> This work doesn't pay.,1.0,This work doesn't pay. <end>
677,This is excellent uine.,<start> This is excellent wine.,1.0,This is excellent wine. <end>
20,I'm tireding.,<start> I'm tired.,1.0,I'm tired. <end>
47,What is the advantage off this technology?,<start> What is the advantage of this technology?,1.0,What is the advantage of this technology? <end>
230,A jack of al trades is a master of none.,<start> A jack of all trades is a master of none.,1.0,A jack of all trades is a master of none. <end>
...,...,...,...,...
709,That land belongs too Mr Ikeda.,<start> This land belongs to Mr Ikeda.,1.0,This land belongs to Mr Ikeda. <end>
491,Dick have a traffic accident.,<start> Dick had a traffic accident.,1.0,Dick had a traffic accident. <end>
19,Nobody cames.,<start> Nobody came.,1.0,Nobody came. <end>
504,It's going to rain.,<start> It's going to rain.,1.0,It's going to rain. <end>


In [ ]:
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),100,replace= False,)]
df_test

,enc_input,dec_input,y,dec_output
2354,I wish her a good night.,<start> I wish her a good night.,NaN,I wish her a good night. <end>
4160,He went to the United States to study medicine.,<start> He went to the United States to study ...,NaN,He went to the United States to study medicine...
2186,Ostriches can't fly.,<start> Ostriches can't fly.,NaN,Ostriches can't fly. <end>
4068,"Though this be madness, yet there is method in't.","<start> Though this be madness, yet there is m...",NaN,"Though this be madness, yet there is method in..."
3149,What does TATOEBA mean?,<start> What does TATOEBA mean?,NaN,What does TATOEBA mean? <end>
...,...,...,...,...
4459,I lost the watch my father had given me.,<start> I lost the watch my father had given me.,NaN,I lost the watch my father had given me. <end>
3099,The cost of living in America was rising.,<start> The cost of living in America was rising.,NaN,The cost of living in America was rising. <end>
3160,You must quit smoking cigarettes.,<start> You must quit smoking cigarettes.,NaN,You must quit smoking cigarettes. <end>
2422,I usually get up at eight o'clock.,<start> I usually get up at eight o'clock.,NaN,I usually get up at eight o'clock. <end>


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [ ]:
class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)
        ##  RETURNING THE ENCODER INPUT , DECODER INPUT , AND DECODER OUTPUT
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)
#WE ARE TAKING THE MAXIMUM LENGHT EQUAL TO 35 WHICH IS 99 PERCENTILE OF THE WORD LENGTH DISTRUBUTION
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataloader = Dataloader( batch_size = 512, dataset=train_dataset)
# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataloader = Dataloader(batch_size=512 , dataset=val_dataset)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
## ENDOCER CLASS
class Encoder(tf.keras.layers.Layer):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''
    def __init__(self , vocab_size , embedding_dim , enc_units , input_len):
        super().__init__()
        
        # STATING ALL THE VARIABLES
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_len = input_len
        self.enc_units = enc_units
        
        # INITALIZING EMBEDDING LAYER
        self.embedding = layers.Embedding(input_dim= self.vocab_size,
                                         output_dim = self.embedding_dim,
                                         mask_zero = True,
                                          input_length = self.input_len
                                         )
        # INTIALIZING LSTM LAYER
        self.lstm_bi = layers.Bidirectional(layers.LSTM(units= self.enc_units,return_state = True,return_sequences=True ))
        self.concat1 = layers.Concatenate()
        self.concat2 = layers.Concatenate()
    def call(self,input):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        # CONVERTING INPUT TO EMBEDDED VECTORS
        emb = self.embedding(input)
        # PASSING THROUGH LSTM LAYER
        enc_output , state_h1 , state_c1 ,state_h2 , state_c2 = self.lstm_bi(emb)
        state_h = self.concat1([state_h1,state_h2])
        state_c  = self.concat2(([state_c1,state_c2]))

        return enc_output ,state_h ,state_c 

## DECODER CLASS
class Decoder(tf.keras.layers.Layer):
    def __init__(self,vocab_size , embedding_dim, dec_unit,input_len ):
        super().__init__()
        # INITALIZING ALL THE VARIABLES 
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_len = input_len
        self.dec_unit =dec_unit
        
        
    def build(self,input_shape):
        
        # INITALIZING EMBEDDING AND LSTM LAYER
        self.embedding = layers.Embedding(input_dim = self.vocab_size,
                                          output_dim = self.embedding_dim,
                                         mask_zero=True,
                                         input_length = self.input_len)
        self.lstm = layers.LSTM(units = self.dec_unit,
                               return_sequences=True,
                               return_state=True)
        
    def call(self,input, state):
        # FORMING THE EMBEDDED VECTORS
        emb = self.embedding(input)
        
        # LSTM OUTPUT
        dec_out,state_h,state_c = self.lstm(emb,initial_state = state)
        
        return dec_out,state_h,state_c
  
## DEFINING THE ARCHITECTURE

# INPUT LAYER
enc_input = layers.Input(shape=(35))
enc_output,state_h,state_c =   Encoder(vocab_size= len(tk_inp.word_index)+1  , embedding_dim= 300 ,
                               enc_units=256 ,input_len=35)(enc_input)
## STORING ENCOER STATES IN A VARIABLE
enc_state = [state_h,state_c] 

# INPUT LAYER FOR DECODER
dec_input = layers.Input(shape = (35))
## DECODER LAYER
dec_output,_,_ = Decoder(vocab_size = len(tk_out.word_index)+1  , embedding_dim = 300,
                               dec_unit=512,input_len=35)(dec_input,enc_state)

# DENSE LAYER
dense = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_output)

# MODEL DEFINING
model  = Model(inputs=[enc_input,dec_input],outputs=dense)        

In [ ]:
callback =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/Colab Notebooks/cs2/model_save/bidirectional_train_emb/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/Colab Notebooks/cs2/model_save/bidirectional_train_emb/logs/save",histogram_freq=1)
]
## TRAINNG AND VALIDATION STEPS FOR ONE EPOCH
train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

# COMPILING THE MODEL
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')
## MODEL SUMMARY
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 35)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 35)]         0           []                               
                                                                                                  
 encoder_2 (Encoder)            ((None, 35, 512),    1502236     ['input_5[0][0]']                
                                 (None, 512),                                                     
                                 (None, 512))                                                     
                                                                                            

In [ ]:
history = model.fit(train_dataloader, steps_per_epoch=train_steps,epochs= 50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

Layer Encoder has arguments ['self', 'vocab_size', 'embedding_dim', 'enc_units', 'input_len']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 1.4084

1/1 [==============================] - 24s 24s/step - loss: 1.4084
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 1.4070

1/1 [==============================] - 9s 9s/step - loss: 1.4070
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 1.4054

1/1 [==============================] - 9s 9s/step - loss: 1.4054
Epoch 4/50
1/1 [==============================] - ETA: 0s - loss: 1.4032

...

1/1 [==============================] - 10s 10s/step - loss: 0.9379
Epoch 47/50
1/1 [==============================] - ETA: 0s - loss: 0.9321

1/1 [==============================] - 10s 10s/step - loss: 0.9321
Epoch 48/50
1/1 [==============================] - ETA: 0s - loss: 0.9267

1/1 [==============================] - 10s 10s/step - loss: 0.9267
Epoch 49/50
1/1 [==============================] - ETA: 0s - loss: 0.9211

1/1 [==============================] - 11s 11s/step - loss: 0.9211
Epoch 50/50
1/1 [==============================] - ETA: 0s - loss: 0.9155

1/1 [==============================] - 10s 10s/step - loss: 0.9155


In [ ]:
def predict(ita_text,model):
    
    # forming integer sequences
    seq = tk_inp.texts_to_sequences([ita_text])
    # padding the sequences
    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    
    # generating the output from encoder
    enc_output,state_h,state_c= model.layers[2](seq)
    
    # placeholder for predicted output
    pred = []
    
    input_state = [state_h,state_c]
    # initailizing the vector for inputing to decoder
    current_vec = tf.ones((1,1))
    
    for i in range(20): # for each word in the input
        # passing each word through decoder layer
        dec_output,dec_state_h,dec_state_c = model.layers[3](current_vec , input_state)
        # passing decoder output through dense  layer
        dense = model.layers[4](dec_output)
        # taking argmax and getting the word index and updating the current vector
        current_vec = np.argmax(dense ,axis = -1)
        # updating the decoder states
        input_state = [dec_state_h,dec_state_c]
        # getting the actual word from the vocab
        pred.append(tk_out.index_word[current_vec[0][0]])
        
        # if the actual word is <end> break the loop
        if tk_out.index_word[current_vec[0][0]]=="<end>":
            break
        
    return " ".join(pred)

In [ ]:
import nltk.translate.bleu_score as bleu
# BELU SCORE
BLEU = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 50,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        #print(b)
        BLEU.append(b)
        
    except:
      continue
print(BLEU)
print("BELU = ", np.mean(BLEU))

51it [00:08,  5.70it/s]

[4.3891732353810896e-232, 5.635809992474887e-232, 9.025629002083406e-232, 1.0032743411283238e-231, 7.813508425061864e-232, 1.171778691554733e-231, 1.0032743411283238e-231, 1.0032743411283238e-231, 1.0016022933125248e-231, 9.97486269044271e-232, 8.166726842395623e-232, 8.166726842395623e-232, 3.690840039559348e-232, 6.510101141652543e-232, 9.918892480173173e-232, 2.4655814830110698e-232, 1.0032743411283238e-231, 5.635809992474887e-232, 5.635809992474887e-232, 9.853445011990208e-232, 7.204611228887491e-232, 9.918892480173173e-232, 7.813508425061864e-232, 8.166726842395623e-232, 1.171778691554733e-231, 8.396161215621529e-232, 6.085166479973199e-232, 7.107197028258987e-232, 5.635809992474887e-232, 6.333134513676991e-155, 6.325072941044999e-232, 7.813508425061864e-232, 7.107197028258987e-232, 1.1862177682648818e-231, 9.269981669466712e-232, 7.107197028258987e-232, 3.418291552750845e-232, 1.1039000191006554e-231, 3.418291552750845e-232, 5.092529201164552e-232, 7.813508425061864e-232, 9.29187

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[10])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[10],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[10])

INPUT SENTENCE ===>  He is happy.
PREDICTED SENTENCE ===>  you <end>
ACTUAL SENTENCE ===>  He is happy. <end>


In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("="*50)
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[19],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  I'll be back at 6:30.
ACTUAL SENTENCE ===>  I'll be back at 6:30. <end>
BEAM SEARCH OUTPUT ,  SCORE
('i is you <end>', -9.796676)
('i is is <end>', -9.848365)
('i is the <end>', -10.041749)
